<a href="https://www.kaggle.com/code/themeeemul/simple-resnet50-using-pytorch?scriptVersionId=143491984" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import cv2 
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,models
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
main_path = "/kaggle/input/pets-facial-expression-dataset/Master Folder/"
train_path = main_path+"train/"
val_path = main_path+"valid/"


In [ ]:
a= ImageFolder(root=train_path)
a[400]

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
#         transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

In [ ]:
class PetDataset(Dataset):
    def __init__(self,root_dir,transform=None):
        self.dataset = ImageFolder(root = root_dir,transform=transform)
        self.transform = transform
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self,index):
        image , label  = self.dataset[index]
        samples = {'image':image,'label':label}
        return samples
        

In [ ]:
train_dataset = PetDataset(train_path,data_transforms['train'])
val_dataset = PetDataset(val_path,data_transforms['validation'])

In [ ]:
train_loader = DataLoader(train_dataset,batch_size = 16,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size = 16,shuffle=False)

In [ ]:
print(len(train_loader))

In [ ]:
# Display image and label.
a = next(iter(train_loader))
print(f"Feature batch shape: {a['image'].size()}")
print(f"Labels batch shape: {a['label'].size()}")
img = a['image'][0].squeeze()
label = a['label'][0]
plt.imshow(img.reshape([224,224,3]))
plt.show()
print(f"Label: {label}")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = models.resnet50(weights=True).to(device)
# model

In [ ]:
# Freezing
for param in model.parameters():
    param.requires_grad = False   
    
num_ftrs = model.fc.in_features

model.fc = nn.Sequential(
               nn.BatchNorm1d(num_ftrs),
               nn.Dropout(0.5),
               nn.Linear(2048, 1024),
               nn.ReLU(inplace=True),
               nn.Linear(1024, 512),
               nn.ReLU(inplace=True),
               nn.BatchNorm1d(512),
               nn.Dropout(0.5),
               nn.Linear(512, 4)).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [ ]:
optimizer.param_groups

In [ ]:
def train_val(model,loader,epoch,flag):
    train_pred = []
    train_labels = []
    train_loss = 0
    total_samples = 0
    for i,data in enumerate(loader):
        optimizer.zero_grad()
        image = data['image'].to(device)
        label = data['label'].to(device)
        
        outputs = model(image)
        loss = criterion(outputs,label)
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.item()
        train_pred.append(torch.argmax(outputs,dim = 1).cpu().detach().numpy())
        train_labels.append(label.cpu().detach().numpy())
        total_samples += data['image'].size(0) 
        
    print("Epoch ",flag," : ",epoch,"| Loss: ",(train_loss/total_samples))
        
    return np.concatenate(train_pred).ravel(), np.concatenate(train_labels).ravel(),(train_loss/total_samples)
        
        

In [ ]:
num_epochs = 10
train_preds = []
train_labels = []
train_loss_list = []

val_preds = []
val_labels = []
val_loss_list = []
for i in range(num_epochs):
#     train
#     current_lr = optimizer.param_groups[0]['lr']
#     print("Epoch: ",i, "| Learning Rate : ", current_lr)
    model.train()
    train_preds,train_labels,train_loss = train_val(model,train_loader,i+1,"train")
    train_loss_list.append(train_loss)
    print("Train Accuracy: ", accuracy_score(train_labels,train_preds))
    
    
    model.eval()
    val_preds,val_labels,val_loss = train_val(model,val_loader,i+1,"val")
    val_loss_list.append(val_loss)
    print("Val Accuracy: ", accuracy_score(val_labels,val_preds))
# validation


In [ ]:
plt.plot([i+1 for i in range(num_epochs)],train_loss_list,'g',label='Training Loss')
plt.plot([i+1 for i in range(num_epochs)],val_loss_list,'r',label='Val Loss')
plt.title('Training and Testing loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
total_samples = 0

# # Iterate through the DataLoader and count samples
for batch in train_loader:
    total_samples += batch['image'].size(0)  # batch[0] contains the input data (images)

print("Sample train = ",total_samples)
total_samples = 0
for batch in val_loader:
    total_samples += batch['image'].size(0)  # batch[0] contains the input data (images)
print("Sample val = ", total_samples)

In [ ]:
# def train_model(model, criterion, optimizer, num_epochs=3):
#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch+1, num_epochs))
#         print('-' * 10)

#         for phase in ['train', 'validation']:
#             if phase == 'train':
#                 model.train()
#             else:
#                 model.eval()

#             running_loss = 0.0
#             running_corrects = 0

#             for inputs in train_loader:
                
#                 image = inputs['image'].to(device)
#                 labels = inputs['label'].to(device)

#                 outputs = model(image)
#                 loss = criterion(outputs, labels)

#                 if phase == 'train':
#                     optimizer.zero_grad()
#                     loss.backward()
#                     optimizer.step()

#                 _, preds = torch.max(outputs, 1)
#                 running_loss += loss.item() * image.size(0)
#                 running_corrects += torch.sum(preds == labels.data)

#             epoch_loss = running_loss / total_samples
#             epoch_acc = running_corrects.double() /total_samples

#             print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
#                                                         epoch_loss,
#                                                         epoch_acc))
#     return model

In [ ]:
# model_trained = train_model(model, criterion, optimizer, num_epochs=3)